# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
from pathlib import Path

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,7.99,78,75,2.57,US,1727322155
1,1,hermanus,-34.4187,19.2345,9.89,87,0,3.47,ZA,1727322125
2,2,dullewala,31.8359,71.4362,34.50,35,0,3.83,PK,1727322326
3,3,yellowknife,62.4560,-114.3525,9.67,100,100,5.36,CA,1727322099
4,4,nago,26.5881,127.9761,29.52,70,54,5.94,JP,1727322330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo = True,
    tiles = "OSM",
    color= "City",
    alpha=0.6,        
    hover_cols=['City', 'Humidity']
)
# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 21
ideal_temp_max = 27
ideal_Cloudiness=0
ideal_wind_speed_max = 4.5
# Apply filtering based on ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temp_min) &
    (city_data_df['Max Temp'] <= ideal_temp_max) &
    (city_data_df['Cloudiness'] == ideal_Cloudiness) &
    (city_data_df['Wind Speed'] <= ideal_wind_speed_max)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display the filtered DataFrame
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.03,GF,1727322503
161,161,al kharijah,25.4514,30.5464,22.71,53,0,3.54,EG,1727322656
169,169,kashgar,39.4547,75.9797,23.56,27,0,3.00,CN,1727322672
234,234,moab,38.5733,-109.5498,24.16,22,0,3.70,US,1727322803
243,243,toliara,-23.3500,43.6667,23.28,72,0,1.64,MG,1727322821


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.at[index, 'Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
87,remire-montjoly,GF,4.9167,-52.2667,89,
161,al kharijah,EG,25.4514,30.5464,53,NaN
169,kashgar,CN,39.4547,75.9797,27,NaN
234,moab,US,38.5733,-109.5498,22,NaN
243,toliara,MG,-23.3500,43.6667,72,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:

# Set parameters to search for a hotel
radius = 10000
params = {  
    'categories': 'accommodation.hotel',
    'limit': 1,  # Get the first result
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
al kharijah - nearest hotel: Kharga Hotel
kashgar - nearest hotel: 深航国际酒店
moab - nearest hotel: Kokopelli Inn
toliara - nearest hotel: Ambary
al jawf - nearest hotel: No hotel found
bechar - nearest hotel: Hôtel Madania
shenandoah - nearest hotel: La Quinta Inn & Suites Baton Rouge Denham Springs
nahrin - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
rupert - nearest hotel: No hotel found
santo andre - nearest hotel: Hotel Blue Tree Towers Santo André
kill devil hills - nearest hotel: Mariner Days Inn & Suites
rafsanjan - nearest hotel: هتل جهانگردی


,City,Country,Lat,Lng,Humidity,Hotel Name
87,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
161,al kharijah,EG,25.4514,30.5464,53,Kharga Hotel
169,kashgar,CN,39.4547,75.9797,27,深航国际酒店
234,moab,US,38.5733,-109.5498,22,Kokopelli Inn
243,toliara,MG,-23.3500,43.6667,72,Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo = True,
    tiles = "OSM",
    color= "City",
    alpha=0.6,        
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)
# Display the map plot_1
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)